## Kernel: Python 3.8

In [ ]:
from dotenv import dotenv_values
from datetime import timedelta, datetime

import pandas as pd
import requests
import locale

## Get Authority

In [ ]:
config = dotenv_values(".env") # get values from .env file
locale.setlocale(locale.LC_ALL, "id") # set local language

def login() -> str:
    try:
        url = 'https://api.kampusmerdeka.kemdikbud.go.id/user/auth/login/mbkm'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
        }
        account = {
            'email': config.get('EMAIL'),
            'password': config.get('PASSWORD')
        }
        response = requests.post(url, json=account, headers=headers)
        token = response.json()['data'].get('access_token')
    
    except Exception:
        raise ValueError(response.json()["error"]["message"])
    
    return token, account

(token, account) = login()
headers = dict()
headers["Accept"] = 'application/json'
headers["Authorization"] = f'Bearer {token}'
headers["User-Agent"] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'

print(f'login success')

## 1. Fetch ALL Mitra (Raw & Test)

In [ ]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=10000&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
reports = response.json().get('data')

print(reports)

## 2. Process Data FULL

In [8]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=10000&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
mitra_data = response.json().get('data')

#===== List Initialization
id_position_cols = []
position_name_cols = []
mitra_name_cols = []
activity_name_cols = []
position_quota_cols = []
location_cols = []
activity_type_cols = []
logo_mitra_cols = []

# TODO: EXTRA FIELDS
start_duration_cols = []
end_duration_cols = []
start_registration_cols = []
end_registration_cols = []
credits_count_cols = []
location_code_cols = []
mitra_id_cols = []
certified_cols = []

Nomor_Mitra_cols = []
Nomor_Mitra = 1

for mitra_data_detail in mitra_data:
    #===== fetch data from mitra_data_detail dict in mitra_data list
    id_position = mitra_data_detail["id"]
    position_name = mitra_data_detail["name"]
    mitra_name = mitra_data_detail["mitra_name"]
    activity_name = mitra_data_detail["activity_name"]
    position_quota = mitra_data_detail["total"]
    location = mitra_data_detail["location"]
    activity_type = mitra_data_detail["activity_type"]
    logo_mitra = mitra_data_detail["logo"]

    # TODO: EXTRA FIELDS
    start_duration = mitra_data_detail["start_duration"]
    end_duration = mitra_data_detail["end_duration"]
    start_registration = mitra_data_detail["start_registration"]
    end_registration = mitra_data_detail["end_registration"]
    credits_count = mitra_data_detail["credits_count"]
    location_code = mitra_data_detail["location_kotakab_code"]
    mitra_id = mitra_data_detail["mitra_id"]
    certified = mitra_data_detail["certified"]

    

    #===== Append Data
    Nomor_Mitra_cols.append(Nomor_Mitra)
    id_position_cols.append(id_position)
    position_name_cols.append(position_name)
    mitra_name_cols.append(mitra_name)
    activity_name_cols.append(activity_name)
    position_quota_cols.append(position_quota)
    location_cols.append(location)
    activity_type_cols.append(activity_type)
    logo_mitra_cols.append(logo_mitra)

    # TODO: EXTRA FIELDS
    start_duration_cols.append(start_duration)
    end_duration_cols.append(end_duration)
    start_registration_cols.append(start_registration)
    end_registration_cols.append(end_registration)
    credits_count_cols.append(credits_count)
    location_code_cols.append(location_code)
    mitra_id_cols.append(mitra_id)
    certified_cols.append(certified)

    Nomor_Mitra += 1

#===== Convert Data to DataFrame
mitra_data_df_full = pd.DataFrame({
    "Nomor_Mitra": Nomor_Mitra_cols,
    "ID Posisi": id_position_cols,
    "Nama Posisi": position_name_cols,
    "Nama Mitra": mitra_name_cols,
    "Nama Kegiatan": activity_name_cols,
    "Kuota Posisi": position_quota_cols,
    "Lokasi Bekerja": location_cols,
    "Tipe Aktivitas": activity_type_cols,
    "URL Logo": logo_mitra_cols,

    # TODO: EXTRA FIELDS
    "Mulai Periode": start_duration_cols,
    "Berakhir Periode": end_duration_cols,
    "Mulai Pendaftaran": start_registration_cols,
    "Berakhir Pendaftaran": end_registration_cols,
    "SKS Diberikan": credits_count_cols,
    "Kode Lokasi": location_code_cols,
    "ID Mitra": mitra_id_cols,
    "Kegiatan Bersertifikat": certified_cols
})


## 2. Process Data COMPACT

In [ ]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=10000&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
mitra_data = response.json().get('data')

Nomor_Mitra_cols = []
id_position_cols = []
position_name_cols = []
mitra_name_cols = []
activity_name_cols = []
position_quota_cols = []
location_cols = []
activity_type_cols = []
logo_mitra_cols = []

for mitra_data_detail in mitra_data:
    # fetch data from mitra_data_detail dict in mitra_data list
    id_position = mitra_data_detail["id"]
    position_name = mitra_data_detail["name"]
    mitra_name = mitra_data_detail["mitra_name"]
    activity_name = mitra_data_detail["activity_name"]
    position_quota = mitra_data_detail["total"]
    location = mitra_data_detail["location"]
    activity_type = mitra_data_detail["activity_type"]
    logo_mitra = mitra_data_detail["logo"]

    # Append Data
    id_position_cols.append(id_position)
    position_name_cols.append(position_name)
    mitra_name_cols.append(mitra_name)
    activity_name_cols.append(activity_name)
    position_quota_cols.append(position_quota)
    location_cols.append(location)
    activity_type_cols.append(activity_type)
    logo_mitra_cols.append(logo_mitra)

    Nomor_Mitra


mitra_data_df_compact = pd.DataFrame({
    "Nomor Mitra": Nomor_Mitra,
    "ID Posisi": id_position_cols,
    "Nama Posisi": position_name_cols,
    "Nama Mitra": mitra_name_cols,
    "Nama Kegiatan": activity_name_cols,
    "Kuota Posisi": position_quota_cols,
    "Lokasi Bekerja": location_cols,
    "Tipe Aktivitas": activity_type_cols,
    "URL Logo": logo_mitra_cols,
})


In [ ]:
mitra_data_df_full.info()

In [ ]:
mitra_data_df_compact.info()

In [ ]:
mitra_data_df_full

## 3. Output Files [CSV and EXCEL]

In [9]:
def output_files(df: pd.DataFrame):
    output_types = ["CSV", "Excel"]
    
    print("=== Output File Types ===")
    
    for i in range(len(output_types)):
        format_name = output_types[i]
        print(f'{i+1}. {format_name}')
    
    choose_number = int(input("Pilih format file untuk mengekspor dataframe: ")) - 1

    now = datetime.now()
    # dd-mm-YY H.M
    dt_string = now.strftime("%d-%m-%Y %H.%M")

    if choose_number == 0:
        filename = './' + dt_string + ' - data mitra mbkm.csv'
        df.to_csv(filename, index=False)
    elif choose_number == 1:
        filename = './' + dt_string + ' - data mitra mbkm.xlsx'
        df.to_excel(filename, index=False)
    else:
        raise ValueError("Not a valid input")

    print(f"Creating {output_types[choose_number]} with a file name suffix `- data mitra mbkm`")

output_files(mitra_data_df_full)

=== Output File Types ===
1. CSV
2. Excel
Creating Excel with a file name suffix `- data mitra mbkm`


## 4. Output Files [JSON]

In [7]:
import json

def output_files_json(df: pd.DataFrame):
    now = datetime.now()
    # dd-mm-YY H.M
    dt_string = now.strftime("%d-%m-%Y %H.%M")
    filename = './' + dt_string + ' - data mitra mbkm.json'
    df.to_json(filename, orient="records")

output_files_json(mitra_data_df_full)

## Extra: 1. Time Config

In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()

print("now =", now)

# dd-mm-YY H.M
dt_string = now.strftime("%d-%m-%Y %H.%M")
filename ='./' + dt_string + ' - data mitra mbkm.csv'
print("date and time =", filename)